In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt  
from sklearn.metrics import mean_squared_error
import MySQLdb

from keras.layers import Dropout
from keras.layers import Dense,Input
from keras.models import Sequential,Model
from keras.callbacks import EarlyStopping
from sklearn.linear_model import LinearRegression

import sys
import xlwt
##get data##
import MySQLdb
from sklearn.preprocessing import MinMaxScaler


def transfromData(trainRaw, testRaw,windosSize):  ##Train ratial, train, test
    sc = MinMaxScaler(feature_range = (0, 1))
    #npRaw = np.array(rawData)
    #scaledData = sc.fit_transform(npRaw.reshape(-1,1))  
    #trainData = np.array(scaledData[:trainCount]).reshape(-1,1)##train 
    #testData = np.array(scaledData[-testCount:]).reshape(-1,1)
    
    npTrain = sc.fit_transform(np.array(trainRaw).reshape(-1,1))
    npTest = sc.fit_transform(np.array(testRaw).reshape(-1,1))
    

    X_train, y_train = splitXy(npTrain,windosSize)
    X_test, y_test = splitXy(npTest,windosSize)
    return sc, X_train, y_train, X_test, y_test

def splitXy(data,windosSize):
    windows = []
    for i in range(windosSize, data.shape[0]):
        windows.append(data[i-windosSize:i, 0])
    np.random.shuffle(windows)
    X = []
    y = []
    for i in range(len(windows)):
        X.append(windows[i][:6])
        y.append(windows[i][windosSize-6:])
    X, y = np.array(X), np.array(y)
    #X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X,y
def buildModel():
    dnn_1_input = Input(shape=(6,),name='dnn_1_input')
    dnn_1 = Dense(128,activation="tanh", name='dnn_1')(dnn_1_input)
    dnn_2 = Dense(64,activation="tanh", name='dnn_2')(dnn_1)
    dnn_3 = Dense(32,activation="tanh", name='dnn_3')(dnn_2)
    dnn_4 = Dense(1, activation='tanh', name='dnn_4')(dnn_3)

    model = Model(dnn_1_input,dnn_4)
    model.summary()
    model.compile(loss='mse',
                  optimizer='adam')
    return model
def getStationList(cursor):
    stationList = []
    queryString = "SELECT distinct station from cleandata"
    cursor.execute(queryString)
    results = cursor.fetchall()
    for row in results :
        val = str(row).strip("',()")
        stationList.append(val)
    stationList.remove('富貴角')##only 106少數資料
    return stationList
def Visualize():
    predicted = sc.inverse_transform(regressor.predict(X_test))
    originY = sc.inverse_transform (y_test)
    print("MSE : ["+str(mean_squared_error(predicted, originY))+"]")
    # Visualising the results
    plt.plot(originY[:100], color = 'red', label = 'Real')  
    plt.plot(predicted[:100], color = 'blue', label = 'Predicted ') 
    plt.legend()
    plt.show()
def writeExcelHead(sheet1,epochs,station):
    sheet1.write(0,1,station)
    raw = 1
    for e in range(epochs):
        sheet1.write(raw,0,e+1)
        raw+=1
def fetchData(cursor,station,windosSize):
    queryString = "SELECT * from cleandata where station='"+station+"' and item = 'PM2.5' and date NOT like '2017%'"
    cursor.execute(queryString)
    results = cursor.fetchall()

    trainRawData = []
    for row in results:
        for i in range(3,27):
            trainRawData.append(float(row[i]))
            
    queryString = "SELECT * from cleandata where station='"+station+"' and item = 'PM2.5' and date like '2017%'"
    cursor.execute(queryString)
    results = cursor.fetchall()

    testRawData = []
    for row in results:
        for i in range(3,27):
            testRawData.append(float(row[i]))

    sc, X_train, y_train, X_test, y_test = transfromData(trainRawData,testRawData,windosSize)
    return sc, X_train, y_train, X_test, y_test
def trainLR(sc, X_train, y_train, X_test, y_test):
    reg = LinearRegression().fit(X_train, y_train)
    predicted_LR = sc.inverse_transform(reg.predict(X_test))
    ori_y_test = sc.inverse_transform(y_test)
    mse = mean_squared_error(predicted_LR, ori_y_test)
    print(mse)
    return mse

Using TensorFlow backend.


In [2]:
import datetime
db=MySQLdb.connect(host="localhost",user="root", passwd="swater0", db="airdb", charset="utf8")
cursor = db.cursor()

stationList = getStationList(cursor)
col=1
for station in stationList:
    print("training : " +station)
    mseList = []
    for windowSize in range(7,17):
        sc, X_train, y_train, X_test, y_test = fetchData(cursor,station,windowSize)
        mse = trainLR(sc, X_train, y_train, X_test, y_test)
        mseList.append(mse)
    book = xlwt.Workbook(encoding="utf-8")
    sheet1 = book.add_sheet("Sheet1")
    writeExcelHead(sheet1,10,station)
    row = 1
    for mse in mseList:
        sheet1.write(row,1,mse)
        row+=1
    book.save("excelFiles/LR/LRresult"+station+".xls")
db.close()


training : 二林
4.113060150621416
12.841878334716634
26.801390477202023
43.36534840677292
62.22726364533793
82.87533296638998
100.73713066731607
115.25818266482747
125.62872670551447
134.2490300409498
training : 南投
2.5957591154347788
10.107533401603405
21.298921129338808
35.514159150588505
52.14774702069974
70.63109405137855
88.05243174550355
101.89926236741508
113.32109423239352
122.77996208675692
training : 埔里
2.140413210210108
9.217428728707942
20.24471321319813
34.73613884746263
52.14640546470152
71.94320902067606
91.58008317207505
107.93300520187877
121.65472826422877
133.01940642935932
training : 大里
4.834580368467631
14.055522243226426
28.41197364480028
44.317657056634
62.107751984305956
81.29614623318233
96.79179692402438
108.87883847143058
116.6475388633803
123.61169017948431
training : 彰化
7.264992797843342
21.63979612362797
44.90527979110377
71.38725834962436
101.34321132830898
133.6040719148551
160.11249332480705
180.56800348536186
193.11385737311412
203.44485488764425
training

78.07995180555692
89.40128821769095
99.02675381490779
107.13974898069999
training : 苗栗
3.843202518855749
10.694071070757039
19.966864109112617
31.09739748972217
43.57163802120591
57.10318201849955
67.61226069917437
75.80080420443119
82.10721731497547
87.05318432737364
training : 頭份
2.2908047918478416
8.4887861489547
17.311805127622893
28.22598433653981
40.6326623367214
54.18728603326025
66.34700563692162
75.2925181026474
82.17620084220538
87.4271349537391
training : 臺東
0.7930663512081058
2.547689814735593
4.5799638668367235
6.848545966031051
9.356826099494837
12.087408372247252
14.221853465588774
15.582630213535973
16.81858447373874
17.972957732581577
training : 花蓮
1.5669388159806426
4.999971128291003
9.179380896979467
13.965102012427607
19.21074602513625
24.85221277763274
29.19981482983063
31.89433188755649
34.02808808053632
35.71674051816952
training : 關山
0.6151149615289826
2.0633838463322554
3.912132566303788
6.085244947164411
8.489865213671038
11.064120357278334
13.166358126516139
